In [1]:
import dask
import dask.dataframe as ddf
import pandas as pd
import numpy as np
import collections
from glob import glob

/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [2]:
path = '/home/ubuntu/scratch/xinrui/intermediate-result-new-select-editors/warring-camps-threshold-100/words-added-removed-all-revisions/'

In [3]:
all_csvs = glob(path + '*')

In [5]:
df = pd.DataFrame()

In [6]:
for i in all_csvs:
    tmp = pd.read_csv(i, sep="\t",encoding='utf-16')
    df = df.append(tmp)

<ipython-input-6-40573b3919d9>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmp)
<ipython-input-6-40573b3919d9>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmp)
<ipython-input-6-40573b3919d9>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmp)
<ipython-input-6-40573b3919d9>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmp)
<ipython-input-6-40573b3919d9>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(tmp)
<ipython-input-6-40573b3919d9>:3: FutureWarning: The fr

In [7]:
df = df.drop(columns={'Unnamed: 0'})

In [8]:
df = df.reset_index().drop(columns={'index'})

In [9]:
def remove_counter(x):
    return x[8:-1]

In [10]:
df['added_words'] = df['added_words'].map(remove_counter)

In [11]:
df['removed_words'] = df['removed_words'].map(remove_counter)

In [13]:
import ast

In [14]:
def convert_to_dict(a):
    if a == '':
        return collections.Counter()
    return collections.Counter(ast.literal_eval(a))

In [15]:
df['added_words'] = df['added_words'].map(convert_to_dict)

In [16]:
df['removed_words'] = df['removed_words'].map(convert_to_dict)

In [17]:
fin = df.reset_index().groupby('group').agg({'added_words':'sum','removed_words':'sum'})

In [18]:
fin['add.removed'] = fin['added_words'] - fin['removed_words']

fin['removed.add'] = fin['removed_words'] - fin['added_words']

In [19]:
def to_negative(x):
    for k,v in x.items():
        x[k] = -v
    return x

In [20]:
fin['removed.add'] = fin['removed.add'].apply(to_negative)

In [21]:
def find_diff(x):
    a = x['add.removed']
    b = x['removed.add']
    return collections.Counter({x: a.get(x, 0) + b.get(x, 0) for x in set(a).union(b)})

In [22]:
fin['diff'] = fin.apply(find_diff,axis=1)

In [23]:
fin = fin[['added_words','removed_words','diff']]

In [24]:
def sort_and_select_top_100(a):
    d= {}
    for k, v in sorted(a.items(), key=lambda x: abs(x[1]), reverse=True):
        d[k] = v
    return list(d.items())[:100]

In [25]:
fin['diff'] = fin['diff'].apply(sort_and_select_top_100)

In [26]:
def sort_counter(c):
    return c.most_common()[:100]

In [27]:
fin['added_words'] = fin['added_words'].apply(sort_counter)

In [28]:
fin['removed_words'] = fin['removed_words'].apply(sort_counter)

In [30]:
fin.to_csv('result/added-removed-words-per-group-all-revisions.csv',sep='\t',encoding='utf-16')